In [115]:
import pandas as pd

In [116]:
PATH = r'transaction_history.csv'
df = pd.read_csv(PATH)

In [117]:
# Rename right value from Method column to 'Purchase of NC coins'
con_1 = (df.Method == "Swap Exact Token...") & (df.From == '0x78e16d2facb80ac536887d1376acd4eeedf2fa08')
con_2 = (df.Method == "Swap")
con_3 = (df.Method == "Swap ETH For Exa...")
con_4 = (df.Method == "Swap Exact ETH F...") 
con_5 = (df.Method == "0x415565b0") 
df.loc[con_1 | con_2 | con_3 | con_4 | con_5, 'Method'] = 'Purchase of NC coins'

# Rename right value from Method column to 'Sales of NC coins'
con_1 = (df.Method == "Swap Exact Token...") & (df.From != '0x78e16d2facb80ac536887d1376acd4eeedf2fa08')
con_2 = (df.Method == "0x0773b509")
df.loc[con_1 | con_2, 'Method'] = 'Sales of NC coins'

# Rename 'Add Liquidity ET...' to 'Add Liquidity'
con_1 = (df.Method == "Add Liquidity ET...")
df.loc[con_1, 'Method'] = 'Add Liquidity'

# Rename 'Remove Liquidity...' to 'Remove Liquidity'
con_1 = (df.Method == "Remove Liquidity...")
df.loc[con_1, 'Method'] = 'Remove Liquidity'

# Remove useless value form Method column
value_to_remove = df.loc[(df.Method == "Stake") | (df.Method == "Fund") | (df.Method == "0xf574133c") | (df.Method == "0x6d9cec22") | (df.Method == "Claim") | (df.Method == "Unstake")].index
df = df.drop(value_to_remove, axis=0)

# Remove useless columns
df = df.drop(["Unnamed: 0", "Unnamed: 4", "Unnamed: 7"], axis=1)

# Format to date type
df["Date Time (UTC)"] = pd.to_datetime(df["Date Time (UTC)"])

In [118]:
df.head()

,Txn Hash,Method,Date Time (UTC),From,To,Quantity
0,0x51725d5786799af2979d05bd8d20d797c9e73406ec4b...,Add Liquidity,2022-01-25 19:48:17,0xb1f0937dc96b679d6a3681108ffbb855c27afb9a,0x78e16d2facb80ac536887d1376acd4eeedf2fa08,6650.000000
2,0xa0866843472868546b2adc2b62325d5fee69fd218333...,Transfer,2022-01-24 14:36:00,0xba93f37118f9d0ac9f620cd7bfb9fd79635db7d3,0x94b7151c0b11daf1a6809f0e56c190ec38fe12d4,190000.000000
4,0x1d2e04ad45f2145fcfcab3ac3b7bf482e03fd7aca1c2...,Sales of NC coins,2022-01-19 14:40:01,0x0aa413b93bf8af1440b7b37ce9bbb75e6a46b970,0x78e16d2facb80ac536887d1376acd4eeedf2fa08,917252.373073
6,0x75411962e2e6527f5a032198816cafe4e1a475a4ebdf...,Add Liquidity,2022-01-16 19:16:02,0xb1f0937dc96b679d6a3681108ffbb855c27afb9a,0x78e16d2facb80ac536887d1376acd4eeedf2fa08,37929.272725
7,0x27f61026e9df4c0c14c6259f624917a12ce7f6c20eb7...,Purchase of NC coins,2022-01-16 19:14:56,0x78e16d2facb80ac536887d1376acd4eeedf2fa08,0xb1f0937dc96b679d6a3681108ffbb855c27afb9a,50814.040553


In [119]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 694 entries, 0 to 710
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Txn Hash         694 non-null    object        
 1   Method           694 non-null    object        
 2   Date Time (UTC)  694 non-null    datetime64[ns]
 3   From             694 non-null    object        
 4   To               694 non-null    object        
 5   Quantity         694 non-null    float64       
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 38.0+ KB


In [120]:
print(f'Unique From Adress {len(df.From.unique())}')
print(f'Unique To Adress {len(df.To.unique())}')

Unique From Adress 43
Unique To Adress 92


# Who add to liquidity pool NC coin?

In [121]:
adresses = df.loc[df.Method == "Add Liquidity", 'From'].value_counts().index
adresses = adresses.tolist()
value_list = []
num_transaction = []

for adress in adresses:
    value_total = df.loc[(df.Method == "Add Liquidity") & (df.From == adress), 'Quantity'].sum()
    value_list.append(round(value_total, 2))
    trans_total = len(df.loc[(df.Method == "Add Liquidity") & (df.From == adress)].value_counts())
    num_transaction.append(trans_total)
    
d = {'Adress': adresses,
    'Number of transaction': num_transaction,
    'Value': value_list}

pd.DataFrame(data=d, columns=['Adress',  'Number of transaction', 'Value'])

,Adress,Number of transaction,Value
0,0xefeea6f3a1bc5b165dcd855a69fa6f20945fbf92,12,333743.67
1,0xba93f37118f9d0ac9f620cd7bfb9fd79635db7d3,10,2392287.25
2,0x817e16b222e0b88bb1dc781445f1a0dba4696f9f,5,882794.98
3,0x66b42b2f46a9c735bb318af2ad3c4460417f2eb8,5,8355.18
4,0x98edd7629f157640b256b2035fe39f8624ebd0d8,4,1458762.35
5,0x7d4490ff7b8ee2a66e2c11f0a69a6916ac07d260,4,565247.94
6,0x407344d866a42ba5ae4357bf6b2f0f2ec475e1a1,4,979009.16
7,0x94b7151c0b11daf1a6809f0e56c190ec38fe12d4,4,221176.91
8,0xb1f0937dc96b679d6a3681108ffbb855c27afb9a,4,1287604.99
9,0x0aa413b93bf8af1440b7b37ce9bbb75e6a46b970,3,52184.12


# Who buy the most NC coin?

In [122]:
adresses = df.loc[df.Method == "Purchase of NC coins", 'To'].value_counts().index
adresses = adresses.tolist()
value_list = []
num_transaction = []

for adress in adresses:
    value_total = df.loc[(df.Method == "Purchase of NC coins") & (df.To == adress), 'Quantity'].sum()
    value_list.append(round(value_total, 2))
    trans_total = len(df.loc[(df.Method == "Purchase of NC coins") & (df.To == adress)].value_counts())
    num_transaction.append(trans_total)
    
d = {'Adress': adresses,
    'Number of transaction': num_transaction,
    'Value': value_list}

pd.DataFrame(data=d, columns=['Adress',  'Number of transaction', 'Value'])

,Adress,Number of transaction,Value
0,0xba93f37118f9d0ac9f620cd7bfb9fd79635db7d3,158,3097272.66
1,0x96c195f6643a3d797cb90cb6ba0ae2776d51b5f3,114,10298.44
2,0xdb6f1920a889355780af7570773609bd8cb1f498,114,3770067.17
3,0x7de553b7bcbcb1e43765d250b9f9f891e1c1fb89,17,1769657.90
4,0x817e16b222e0b88bb1dc781445f1a0dba4696f9f,5,249727.26
5,0x407344d866a42ba5ae4357bf6b2f0f2ec475e1a1,4,558057.01
6,0x722bf7ccf922974976272d7f6795e0caac7cf661,3,157292.09
7,0x4b36c745993ebd4a004e73112b3670f680a478a6,3,101707.22
8,0xb1f0937dc96b679d6a3681108ffbb855c27afb9a,3,409051.29
9,0xa17f7de14ee47efb1c822a7610781c3679bcdf92,2,882.66


#  Who sold the most NC coin? 

In [123]:
adresses = df.loc[df.Method == "Sales of NC coins", 'From'].value_counts().index
adresses = adresses.tolist()
value_list = []
num_transaction = []

for adress in adresses:
    value_total = df.loc[(df.Method == "Sales of NC coins") & (df.From == adress), 'Quantity'].sum()
    value_list.append(round(value_total, 2))
    trans_total = len(df.loc[(df.Method == "Sales of NC coins") & (df.From == adress)].value_counts())
    num_transaction.append(trans_total)
    
d = {'Adress': adresses,
    'Number of transaction': num_transaction,
    'Value': value_list}

pd.DataFrame(data=d, columns=['Adress',  'Number of transaction', 'Value'])

,Adress,Number of transaction,Value
0,0xba93f37118f9d0ac9f620cd7bfb9fd79635db7d3,40,1698362.20
1,0x7d4490ff7b8ee2a66e2c11f0a69a6916ac07d260,8,638752.90
2,0x7de553b7bcbcb1e43765d250b9f9f891e1c1fb89,5,641199.35
3,0x0aa413b93bf8af1440b7b37ce9bbb75e6a46b970,2,955252.37
4,0x4e9abecb9c36f211a8ffc958cbc0ef05cbdeea05,1,116850.00
5,0x407344d866a42ba5ae4357bf6b2f0f2ec475e1a1,1,383023.41
6,0x98edd7629f157640b256b2035fe39f8624ebd0d8,1,475000.00
7,0x7c279629c60385474629f72ae1c0ce0f93e08277,1,91666.41
8,0x5d852cd818aaee3fdadb880fc46bd2be60527d87,1,129.08


#  Who collect the biggest Airdrop - top20? 

In [124]:
adresses = df.loc[(df.Method == "Transfer") & (df.From == '0xba93f37118f9d0ac9f620cd7bfb9fd79635db7d3'), 'To'].value_counts().index
adresses = adresses.tolist()
value_list = []

for adress in adresses:
    value_total = df.loc[(df.Method == "Transfer") & (df.To == adress), 'Quantity'].sum()
    value_list.append(round(value_total, 2))
    
d = {'Adress': adresses,

    'Value': value_list}

temp_df = pd.DataFrame(data=d, columns=['Adress', 'Value'])
temp_df.sort_values('Value', ascending=False)[:20]

,Adress,Value
9,0x98edd7629f157640b256b2035fe39f8624ebd0d8,2090000.0
4,0x7d4490ff7b8ee2a66e2c11f0a69a6916ac07d260,1947500.0
14,0x90d4f1bd47d26743d93c61525f22ba1276c24616,1235000.0
1,0xefeea6f3a1bc5b165dcd855a69fa6f20945fbf92,1187500.0
15,0x0aa413b93bf8af1440b7b37ce9bbb75e6a46b970,1059250.0
2,0x4b36c745993ebd4a004e73112b3670f680a478a6,1045000.0
30,0x407344d866a42ba5ae4357bf6b2f0f2ec475e1a1,1000000.0
16,0x7de553b7bcbcb1e43765d250b9f9f891e1c1fb89,997500.0
32,0xcab0ab3416f162f152492b9ea722a4a178167e0f,969000.0
17,0x06448d32374560d858ba016e6bf16fa135f6e075,950000.0


In [137]:
x = adresses = df.loc[(df.Method == "Transfer") & (df.From == '0xba93f37118f9d0ac9f620cd7bfb9fd79635db7d3'), 'To']
print(f'Total number of Airdrop: {len(x)}')

Total number of Airdrop: 106


#  Who sell the  Airdrop? 

In [136]:
adresses_airdrop = df.loc[(df.Method == "Transfer") & (df.From == '0xba93f37118f9d0ac9f620cd7bfb9fd79635db7d3'), 'To'].value_counts().index
adresses_airdrop = adresses_airdrop.tolist()
seller_adresses = df.loc[(df.Method == "Sales of NC coins"), 'From']
seller_adresses = seller_adresses.tolist()
airdrop_seller_list = []

for seller_adress in seller_adresses:
    if seller_adress in adresses_airdrop:
        airdrop_seller_list.append(seller_adress)

pd.DataFrame(data=airdrop_seller_list, columns=['Adress'])

,Adress
0,0x0aa413b93bf8af1440b7b37ce9bbb75e6a46b970
1,0x4e9abecb9c36f211a8ffc958cbc0ef05cbdeea05
2,0x407344d866a42ba5ae4357bf6b2f0f2ec475e1a1
3,0x98edd7629f157640b256b2035fe39f8624ebd0d8
4,0x7d4490ff7b8ee2a66e2c11f0a69a6916ac07d260
5,0x7d4490ff7b8ee2a66e2c11f0a69a6916ac07d260
6,0x7d4490ff7b8ee2a66e2c11f0a69a6916ac07d260
7,0x7d4490ff7b8ee2a66e2c11f0a69a6916ac07d260
8,0x7de553b7bcbcb1e43765d250b9f9f891e1c1fb89
9,0x7d4490ff7b8ee2a66e2c11f0a69a6916ac07d260
